In [2]:
import numpy as np
np.set_printoptions(linewidth = 200, precision = 4, suppress = True)

In [3]:
data = np.loadtxt('../NC-Data.csv', delimiter=',', dtype=str)
data = data[1:].astype(float)
data

array([[ 1.    , -0.0461,  0.2312, ...,  0.2704,  0.4664,  0.3672],
       [-0.0461,  1.    , -0.0671, ..., -0.0515, -0.0944, -0.0349],
       [ 0.2312, -0.0671,  1.    , ...,  0.147 ,  0.2608,  0.6313],
       ...,
       [ 0.2704, -0.0515,  0.147 , ...,  1.    ,  0.2066,  0.1538],
       [ 0.4664, -0.0944,  0.2608, ...,  0.2066,  1.    ,  0.3486],
       [ 0.3672, -0.0349,  0.6313, ...,  0.1538,  0.3486,  1.    ]])

In [4]:
nodes = np.loadtxt('../NC-K7-Trace-Nodes.csv', delimiter=',', dtype=str)
nodes = nodes[1:].astype(float)
bounds = np.loadtxt('../NC-K7-Trace-Bounds.csv', delimiter=',', dtype=str)
bounds = bounds[1:].astype(float)

In [5]:
nodes = np.loadtxt('../NC-K7-Trace-Nodes.csv', delimiter=',', dtype=str)
nodes = nodes[1:].astype(float)
bounds = np.loadtxt('../NC-K7-Trace-Bounds.csv', delimiter=',', dtype=str)
bounds = bounds[1:].astype(float)

In [6]:
# One lower bound impl, and two upper bound impls
bounds[:, 60]

array([4.5265, 6.1343, 7.    ])

In [7]:
selected = np.where(nodes[:, 60] == 1)[0]
selected

array([57, 82, 83, 84, 85])

In [8]:
selected = np.r_[selected, [86]]
selected

array([57, 82, 83, 84, 85, 86])

In [9]:
data[selected, :][:, selected]

array([[1.    , 0.4014, 0.3903, 0.3946, 0.3465, 0.3374],
       [0.4014, 1.    , 0.9643, 0.9311, 0.95  , 0.0515],
       [0.3903, 0.9643, 1.    , 0.9818, 0.9886, 0.0701],
       [0.3946, 0.9311, 0.9818, 1.    , 0.9772, 0.0973],
       [0.3465, 0.95  , 0.9886, 0.9772, 1.    , 0.0642],
       [0.3374, 0.0515, 0.0701, 0.0973, 0.0642, 1.    ]])

In [10]:
np.sqrt(np.linalg.eigvalsh(data[selected, :][:, selected]))

array([0.0894, 0.1366, 0.2687, 0.7712, 1.0972, 2.0255])

In [41]:
# Use "raw" data SVD from here on out, singular values are slightly different.
np.random.seed(1)
n_obs = 100
raw_data = np.random.multivariate_normal(
    np.zeros_like(selected, float),
    data[selected, :][:, selected],
    size=n_obs)
raw_data.shape

(100, 6)

In [42]:
np.linalg.svd(raw_data)[1]

array([19.7849, 12.0553,  7.1674,  2.6202,  1.3461,  0.862 ])

In [43]:
np.linalg.svd(np.c_[raw_data, raw_data])[1][::-1] / np.sqrt(2)

array([ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.862 ,  1.3461,  2.6202,  7.1674, 12.0553, 19.7849])

In [44]:
v = np.linalg.svd(raw_data[:, 0:5])[2][0:1, :].T
v

array([[0.2481],
       [0.4778],
       [0.4927],
       [0.4757],
       [0.4911]])

In [45]:
import scipy.linalg
mat = scipy.linalg.block_diag(v @ v.T, [[0]], np.eye(5) - (v @ v.T), [[1]])
rotation_matrix_small = scipy.linalg.block_diag(
    np.linalg.svd(raw_data[:, 0:5])[2].T,
    [[1]])
# mat = mat @ scipy.linalg.block_diag(rotation_matrix_small, np.eye(6))

In [49]:
np.square(np.linalg.svd(np.c_[raw_data, raw_data] @ mat)[1][::-1])

array([  0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.7431,   1.812 ,   6.8655,  51.3719, 145.3307, 391.4425])

In [51]:
# Gershgorin circle theorem after projecting onto two different matrices.
# We sparsified data_expanded, but Gershgorin bound is performing poorly!
apply_mat = np.c_[raw_data, raw_data] @ mat
data_expanded = apply_mat.T @ apply_mat
np.sort(abs(data_expanded).sum(axis=1))

array([  0.    ,  22.6359,  23.5315,  25.3234,  32.1583, 189.4509, 219.9754, 305.3621, 421.8053, 423.71  , 435.4915, 436.9155])

In [52]:
# Gershgorin circle theorem (no change of variables).
np.sort(abs(raw_data.T @ raw_data).sum(axis=1))

array([257.8039, 312.7489, 411.7866, 413.0769, 420.3779, 425.2604])

In [53]:
abs(data_expanded).sum(axis=1)

array([219.9754, 423.71  , 436.9155, 421.8053, 435.4915,   0.    , 189.4509,  25.3234,  23.5315,  22.6359,  32.1583, 305.3621])

In [54]:
data_expanded

array([[ 23.329 ,  44.9357,  46.3362,  44.7337,  46.1852,   0.    ,  -0.    ,   0.    ,  -0.    ,   0.    ,   0.    ,  14.4556],
       [ 44.9357,  86.5538,  89.2514,  86.1647,  88.9605,   0.    ,  -0.    ,   0.    ,  -0.    ,   0.    ,   0.    ,  27.8439],
       [ 46.3362,  89.2514,  92.033 ,  88.8502,  91.7331,   0.    ,  -0.    ,   0.    ,  -0.    ,   0.    ,   0.    ,  28.7117],
       [ 44.7337,  86.1647,  88.8502,  85.7774,  88.5606,   0.    ,  -0.    ,   0.    ,  -0.    ,   0.    ,   0.    ,  27.7187],
       [ 46.1852,  88.9605,  91.7331,  88.5606,  91.4341,   0.    ,  -0.    ,   0.    ,  -0.    ,   0.    ,   0.    ,  28.6181],
       [  0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ],
       [ -0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ,   0.    ,  94.6169,  -9.6804, -11.6543, -10.2479, -16.7561,  46.4953],
       [  0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,  -9.6804,   5.8146,   0.5809,

In [55]:
data_vector = np.linalg.svd(raw_data)[0][:, 0:1]

In [56]:
data_proj = data_vector @ data_vector.T
data_proj_compl = np.eye(n_obs) - data_proj
raw_data_new = np.c_[data_proj @ raw_data, data_proj_compl @ raw_data]

In [57]:
np.square(np.linalg.svd(raw_data)[1])

array([391.4425, 145.3307,  51.3719,   6.8655,   1.812 ,   0.7431])

In [62]:
np.square(np.linalg.svd(raw_data_new)[1]).max()

391.4424958109583

In [59]:
abs(raw_data_new.T @ raw_data_new).sum(axis=1)

array([254.5931, 427.197 , 441.1269, 426.9439, 438.7903, 190.8679, 195.5678,  47.4856,  49.5438,  43.6973,  58.6174, 192.4244])

In [60]:
raw_data_new.T @ raw_data_new

array([[ 29.7394,  49.9016,  51.5287,  49.872 ,  51.2558,  22.2956,   0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ,   0.    ],
       [ 49.9016,  83.7328,  86.4631,  83.6832,  86.0052,  37.4111,   0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ],
       [ 51.5287,  86.4631,  89.2825,  86.4119,  88.8096,  38.631 ,   0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ,   0.    ],
       [ 49.872 ,  83.6832,  86.4119,  83.6336,  85.9542,  37.3889,   0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ],
       [ 51.2558,  86.0052,  88.8096,  85.9542,  88.3392,  38.4264,  -0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ],
       [ 22.2956,  37.4111,  38.631 ,  37.3889,  38.4264,  16.715 ,   0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ,   0.    ],
       [  0.    ,   0.    ,   0.    ,   0.    ,  -0.    ,   0.    ,  88.2065, -14.6462, -16.8468, -15.3862, -21.8267,  38.6553],
       [ -0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    ,  -0.    , -14.6462,   8.6356,   3.3692,

In [61]:
np.linalg.norm((raw_data_new.T @ raw_data_new)[0:6, 0:6], ord=2)

391.44249581095835